# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [4]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [5]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [6]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [7]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [10]:
def foldl2(f, x0, lst):
    return foldr(lambda val, acc: lambda n: acc(f(n, val)), lambda x: x, lst)(x0)

In [11]:
#сумма
foldl2(lambda x, y: x + y, 0, [1, 2, 3])

6

In [12]:
#конкатенация
foldl2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [13]:
f = lambda x, y: x / y
foldl2(f, 1, [1, 2, 3])

0.16666666666666666

In [14]:
def foldr2(f, x0, lst):
    return foldl(lambda acc, val: lambda n: acc(f(val, n)), lambda x: x, lst)(x0)

In [15]:
#сумма
foldr2(lambda x, y: x + y, 0, [1, 2, 3])

6

In [18]:
#конкатенация
foldr2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [19]:
f = lambda x, y: x / y
foldr2(f, 1, [1, 2, 3])

1.5


Задача 2

In [1]:
def check_inv(a, b):
    if len(b) == 0 : return True
    for i in range(0, len(a), len(b)):
        for j in range(0, len(b)):
            if a[i] == b[j]: 
                return check_inv(a[0:i] + a[i+1:], b[0:j] + b[j+1:])
    return False

In [2]:
a = 'abcrotm'

b = 'tro'

check_inv(a, b)

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [26]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if(self.left == None and self.right == None):
            yield self.value
            
        if(self.left):
            yield from self.left
            
        if(self.right):
            yield from self.right
        
    
    def __str__(self):
        res = [str(self.value)]
        
        if(self.left):
            res.append(self.left.__str__())
            
        if(self.right):
            res.append(self.right.__str__())
            
        return ' '.join(res)
    
    def __repr__(self):
        if(self.left == None and self.right == None):
            return f"Tree({self.value})"
        
        if(self.right == None):
            return f"Tree({self.value}, {self.left.__repr__()})"
        
        if(self.left == None):
            return f"Tree({self.value}, None, {self.right.__repr__()})"
        
        
        return f"Tree({self.value}, {self.left.__repr__()}, {self.right.__repr__()})"
 
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

In [27]:
# __str__ test
print(tree)

0 1 3 4 2


In [28]:
# __repr__ test
tree

Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

In [29]:
# __repr__ eval test
tree_repr_test = eval(tree.__repr__())
print(tree_repr_test)

0 1 3 4 2


In [30]:
# __iter__ test
for it in tree:
    print(it)

3
4
2


# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [1]:
def parse(expr):
    buf = []

    for val in expr.split(" "):
        if(val[0] == "("):
            buf.append("(")
            buf.append(val[1:])
        elif(val[-1] == ")"):
            buf.append(val[:-1])
            buf.append(")")
        else:
            buf.append(val)

    result = []
    stack = []
    high_priority = set(["*", "/"])
    low_priority = set(["+", "-"])
    
    for val in buf:
        if(val.isnumeric()):
            result.append(val)
        if(val == "("):
            stack.append(val)
        if(val == ")"):
            while(stack[-1] != "("):
                result.append(stack.pop())
            stack.pop()
        if(val in low_priority):
            while(len(stack) > 0 and stack[-1] in high_priority):
                result.append(stack.pop())
            stack.append(val)
        if(val in high_priority):
            stack.append(val)
            
    while(len(stack) > 0):
        result.append(stack.pop())
    
    return result

def calc(expr):
    operations = {
        "+": lambda x, y: x + y,
        "-": lambda x, y: x - y,
        "*": lambda x, y: x * y,
        "/": lambda x, y: x / y
    }
    
    expr = parse(expr)
    stack = []
    
    for val in expr:
        if(val.isnumeric()):
            stack.append(int(val))
        if(val in operations):
            right = stack.pop()
            left = stack.pop()
            stack.append(operations[val](left, right))
    return stack.pop()

calc('2 * (15 - 3 * 4) - 2') == 4

True